TFIM
1. 緩和時間を指数関数でフィッティング tfim_tau_hx_n{nstamp}

In [9]:
import qutip as qt
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import lru_cache
import requests
# 通知設定
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1439055708988375082/6iiXA8J_3Bn1RPWZ8XmPfmfSJp_PkGdaIzt495Ao2fbu_a09VnMJpxmrnSyfN4Wtyc7T"
def notify_discord(message: str):
    payload = {"content": message}
    try:
        r = requests.post(DISCORD_WEBHOOK_URL, json=payload, timeout=5)
        r.raise_for_status()
        print("✅ Discord 通知送信 OK")
    except Exception as e:
        print("⚠ Discord 通知に失敗:", e)

# グラフ設定
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('default')
mpl.rcParams.update({
    "axes.labelsize": 18,   # 軸ラベル
    "xtick.labelsize": 16,  # 目盛り数字
    "ytick.labelsize": 16,
    "lines.linewidth": 2.2, # 線の太さ
    "axes.titlesize": 16,
})

def inward_ticks(ax, major_len=7, minor_len=4, width=1.2):
    ax.minorticks_on()
    ax.tick_params(which='both', direction='in', top=True, right=True, length=major_len, width=width)
    ax.tick_params(which='minor', length=minor_len, width=width)

markers = ['o', 's', '^', 'v', 'D', '*', '+', 'x']
colors = [
    "black",
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:purple",
    "tab:red",
    "tab:brown",
    "tab:gray",
]

# hx スイープ用: 小さい hx → 薄い / 大きい hx → 濃い紫
colors_hx = [
    "thistle",        # いちばん薄い
    "plum",
    "violet",
    "orchid",
    "mediumorchid",
    "mediumpurple",
    "purple",
    "indigo",         # いちばん濃い
]

# N スイープ用: N が大きいほど「暖色・濃い」方向
colors_N = [
    "tab:blue",   # 最小 N
    "tab:cyan",
    "tab:green",
    "tab:olive",
    "tab:orange",
    "tab:red",    # 最大 N
    "tab:brown",
    "tab:gray",   # おまけ枠（補助曲線にも使える）
]
OUTDIR = Path("charts")
def save_fig(fig, stem: str):
    fig.savefig(OUTDIR / f"{stem}.pdf", bbox_inches="tight", dpi=300) 
    print(f"saved: {OUTDIR / (stem + '.pdf')}")

# データ保存
DATADIR = Path("data")

# データ読み込み例
# fname = DATADIR / "square_sa_tcut.npz" 

# dat = np.load(fname)

# N_list  = dat["N"]   # shape (nN,)
# hx_list = dat["hx_list"]  # shape (nHx,)
# Tcut_list = dat["Tcut"]
# EE      = dat["EE"]       # shape (nN, nHx)

In [10]:
# ========= キャッシュ付きヘルパ =========
@lru_cache(maxsize=None)
def get_ops(N):
    """Nだけで決まる演算子たち"""
    cut_pos = N // 2 - 1
    sx, sz = [0.5 * M for M in (qt.sigmax(), qt.sigmaz())]
    I2 = qt.qeye(2)

    def op_at(i, op):
        return qt.tensor([op if k == i else I2 for k in range(N)])

    Sx = [op_at(i, sx) for i in range(N)]
    Sz = [op_at(i, sz) for i in range(N)]

    def H_exchange(i):
        return Sz[i] * Sz[i+1]

    Hcut_unit   = H_exchange(cut_pos)
    Hex_rest_unit = sum(H_exchange(i) for i in range(N-1) if i != cut_pos)
    Hhx_unit    = sum(Sx[i] for i in range(N))

    left_sites = list(range(N//2))
    return Hcut_unit, Hex_rest_unit, Hhx_unit, left_sites


@lru_cache(maxsize=None)
def get_psi0(N, J, hx):
    """(N, J, hx)で決まる初期基底状態"""
    Hcut_unit, Hex_rest_unit, Hhx_unit, _ = get_ops(N)
    H_full = -J * (Hcut_unit + Hex_rest_unit) - hx * Hhx_unit
    _, psi0 = H_full.groundstate(sparse=True)
    return psi0

# シミュレーション関数
# ========= メイン: Tcut ごとの時間発展 =========

def simulate(N, J, hx, Tcut):
    """J, hx, Tcutだけで決まるもの"""
    Hcut_unit, Hex_rest_unit, Hhx_unit, left_sites = get_ops(N)
    psi0 = get_psi0(N, J, hx)

    H_static = -J * Hex_rest_unit - hx * Hhx_unit

    def ramp(t, T_cut):
        if T_cut == 0.0:
            # 瞬時切断：t<=0で1, t>0で0（t=0 から見れば状態は変わらない）
            return 1.0 if t <= 0.0 else 0.0
        if t <= 0.0:
            return 1.0
        if t >= T_cut:
            return 0.0
        return 1.0 - t / T_cut

    Ht = [H_static, [-J * Hcut_unit, lambda t, args: ramp(t, Tcut)]]

    if hx == 0.0:
        rhoL = qt.ptrace(psi0, left_sites)
        return float(qt.entropy_vn(rhoL, base=2))

    if Tcut > 0.0:
        tlist = [0.0, float(Tcut)]
    else:
        tlist = [0.0]

    opts = {
        "method": "bdf",
        "nsteps": 100_000_000,
        "rtol": 1e-6,
        "atol": 1e-8,
        "max_step": (Tcut / 500.0) if Tcut > 0.0 else 0.1,
        "store_states": True,
        "progress_bar": None,
    }

    res = qt.sesolve(Ht, psi0, tlist, e_ops=[], options=opts)
    psi_T = res.states[-1]

    rhoL = qt.ptrace(psi_T, left_sites)
    return float(qt.entropy_vn(rhoL, base=2))


In [ ]:
N_list = [10, 12]
J = 1.0
hx_list = [0.1]
Tcut_list = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 1e5, 2e5, 5e5, 1e6, 2e6, 5e6, 1e7, 2e7, 5e7, 1e8, 2e8, 5e8]

hxstamp = '-'.join(map(str, hx_list))

# fname = DATADIR / f"tfim_tau_vs_N_hx{hxstamp}"
# dat = np.load(fname)

# tau_all = dat["tau_all"]

tau_all = []

fig, ax = plt.subplots(figsize=(6.9, 4.6))
inward_ticks(ax)  # これは 1 回でOK

for i, hx in enumerate(hx_list):
    for j, N in enumerate(N_list):
        # t=0 のエントロピーを1回だけ計算
        SA0 = simulate(N=N, J=J, hx=hx, Tcut=0.0)
        print(f"N is {N}")
        tau = None
        for Tcut in Tcut_list:
            print(f"Tcut is {Tcut}")
            SA_Tcut = simulate(N=N, J=J, hx=hx, Tcut=Tcut)
            if SA_Tcut <= SA0 / 2:
                tau = Tcut
                print(f"tau is {tau}")
                break
    
        # 見つからなかった場合の扱い（ここは好みで）
        if tau is None:
            tau = np.nan  # もしくは Tcut_list[-1] とか


# データ
hx_list = [0.1, 0.2, 0.5]
N_list = [2, 4, 6, 8, 10, 12]
tau_list = [[20, 500, 2e4, 5e5, np.nan, np.nan], [10, 50, 200, 2000, 1e4, 5e4], [5, 5, 10, 10, 10, np.nan]]
    
#     # ---- プロット ----
#     ax.plot(
#         N_list,
#         # tau_list,
#         tau_all[i],
#         marker=markers[i],
#         color=colors_hx[i],
#         markersize=6.0,
#         linewidth=2.2,
#     )

# ax.set_xlim(2, 24)
# ax.set_xlabel(r"$N$", fontsize=24)
# ax.set_ylim(1, 1e10)
# ax.set_yscale("log")
# ax.set_ylabel(r"$\tau$", fontsize=24)

# # ---- データ保存 ---
# npz_path = DATADIR / f"tfim_tau_vs_N_hx{hxstamp}"
# np.savez(
#     npz_path,
#     N_list=np.array(N_list),
#     hx_list=np.array(hx_list),
#     Tcut_list=np.array(Tcut_list),
#     tau_all=np.array(tau_all),
# )

# hx = 0.1
# 20 500 2e4 5e5 - - 
# hx = 0.2
# 10 50 200 2000 1e4 5e4
# hx = 0.5
# 5 5 10 10 10 10

notify_discord(message="tfim tau vs N done")
# save_fig(fig=fig, stem=f"tfim_tau_vs_N_hx{hxstamp}")


N is 10
Tcut is 1
Tcut is 2
Tcut is 5
Tcut is 10
Tcut is 20
Tcut is 50
Tcut is 100
Tcut is 200
Tcut is 500
Tcut is 1000
Tcut is 2000
Tcut is 5000
Tcut is 10000
Tcut is 20000
Tcut is 50000
Tcut is 100000.0
Tcut is 200000.0
Tcut is 500000.0
Tcut is 1000000.0
Tcut is 2000000.0
Tcut is 5000000.0
